## Blending inferred blks together..
Blend inferred .nii.gz chunks back together

<img src="Blending.png" width="500">

In [1]:
# Import python library
import numpy as np
import h5py as h5
from skimage import exposure
import os
import nibabel as nib
import tifffile as tf

Define helper functions: 

`read_niigz`: Read specific levels of a .nii.gz file

`save_niigz`: Save stitched volume ot a .nii.gz file

`Blend`: Blend overlapping region using a max blending approach, output a stitched volume.

In [ ]:

def read_niigz(file_path, level=None):
    """
    Read specific levels of a .nii.gz file to avoid overloading memory.
    
    Parameters:
    file_path (str): Path to the .nii.gz file.
    level (int or slice, optional): Specific level or slice to read. Default is None (read full data lazily).
    
    Returns:
    numpy.ndarray: The data of the specified level or slice.
    """
    img = nib.load(file_path)
    proxy = img.dataobj  # Lazy loading proxy

    # Get shape and determine the shortest axis
    shape = proxy.shape
    shortest_axis = np.argmin(shape)

    if level is not None:
        if shortest_axis == 0:
            data = proxy[level, :, :]
        elif shortest_axis == 1:
            data = proxy[:, level, :]
        elif shortest_axis == 2:
            data = proxy[:, :, level]
    else:
        # Load full data lazily
        data = proxy[:]
        # Move the shortest axis to the 3rd position
        axes_order = [i for i in range(len(shape)) if i != shortest_axis] + [shortest_axis]
        data = np.moveaxis(data, axes_order, np.arange(len(shape)))

    return data


def save_niigz(numpy_arr, output_path):
    """Saves a NumPy array as a .nii.gz file."""
    stitched_img = nib.Nifti1Image(numpy_arr, np.eye(4))
    nib.save(stitched_img, output_path)


def Blend(savepath, output_path, blocksize = 1024, overlap_percentage = 0.25):
    # Find all block files in the directory
    block_files = [f for f in os.listdir(savepath) if f.endswith('.nii.gz')]

    # Extract the image shape from one of the blocks
    example_block = read_niigz(os.path.join(savepath, block_files[0]))
    depth = example_block.shape[2]

    # Calculate the number of blocks in x and y directions
    xy_values = [(int(block_file.split('_')[2]), int(block_file.split('_')[3].split('.')[0])) for block_file in block_files]
    x_blocks = max(x for x,y in xy_values) + 1
    y_blocks = max(y for x,y in xy_values) + 1

    # Calculate the dimensions of the final stitched volume
    x_dim = int(blocksize + (x_blocks - 1) * blocksize * (1 - overlap_percentage))
    y_dim = int(blocksize + (y_blocks - 1) * blocksize * (1 - overlap_percentage))

    # Create an empty volume to hold the stitched image
    stitched_volume = np.zeros((x_dim, y_dim, depth), dtype=np.uint16)

    # Iterate over each block file and place it in the stitched volume
    for block_file in block_files:
        i = int(block_file.split('_')[2])
        j = int(block_file.split('_')[3].split(".nii.gz")[0])

        # Calculate start and end indices for x and y dimensions
        x_start = int(i * blocksize * (1 - overlap_percentage))
        x_end = x_start + blocksize
        y_start = int(j * blocksize * (1 - overlap_percentage))
        y_end = y_start + blocksize

        block_data = read_niigz(os.path.join(savepath, block_file))

        # Apply max blending in the overlapping region
        stitched_volume[x_start:x_end, y_start:y_end] = np.maximum(
            stitched_volume[x_start:x_end, y_start:y_end], block_data)

    return stitched_volume

In [ ]:
savepath = "" #specify the path to your inferred .niigz folder
output_path = savepath + os.sep + "stitched"
print(f"Stitching: {output_path}")

stitched_volume = Blend(savepath, 
                        output_path, 
                        blocksize = 1024, 
                        overlap_percentage = 0.25)

# Save the stitched volume
save_niigz(stitched_volume, output_path)
print(f"Stitched volume saved at: {output_path}")

Stitching: /media/W/UPenn_Clinical/OTLS4_NODO_6-9-23_AFM098_well_9/nnunet/results807_ep1500/stitched
